In [1]:
import os
import pandas as pd
import lxml
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = "data/scores"

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [4]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores]

In [5]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
    
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [6]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    line_score = line_score[["team", "total"]]
    return line_score

In [7]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [8]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [9]:
base_cols = None
games = []

for box_score in box_scores:

    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T
    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")


100 / 5634
200 / 5634
300 / 5634
400 / 5634
500 / 5634
600 / 5634
700 / 5634
800 / 5634
900 / 5634
1000 / 5634
1100 / 5634
1200 / 5634
1300 / 5634
1400 / 5634
1500 / 5634
1600 / 5634
1700 / 5634
1800 / 5634
1900 / 5634
2000 / 5634
2100 / 5634
2200 / 5634
2300 / 5634
2400 / 5634
2500 / 5634
2600 / 5634
2700 / 5634
2800 / 5634
2900 / 5634
3000 / 5634
3100 / 5634
3200 / 5634
3300 / 5634
3400 / 5634
3500 / 5634
3600 / 5634
3700 / 5634
3800 / 5634
3900 / 5634
4000 / 5634
4100 / 5634
4200 / 5634
4300 / 5634
4400 / 5634
4500 / 5634
4600 / 5634
4700 / 5634
4800 / 5634
4900 / 5634
5000 / 5634
5100 / 5634
5200 / 5634
5300 / 5634
5400 / 5634
5500 / 5634
5600 / 5634


In [10]:
games_df = pd.concat(games, ignore_index=True)

In [11]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
1,240.0,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
2,240.0,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,...,20.0,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False
3,240.0,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,...,28.6,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True
4,240.0,240.0,35.0,83.0,0.422,9.0,24.0,0.375,12.0,18.0,...,33.3,30.9,171.0,105.0,MIN,107,1,2017,2017-03-08,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11263,240.0,240.0,37.0,89.0,0.416,12.0,27.0,0.444,13.0,15.0,...,37.5,30.0,170.0,120.0,SAS,98,0,2017,2017-04-10,True
11264,240.0,240.0,34.0,95.0,0.358,7.0,33.0,0.212,17.0,23.0,...,59.2,34.5,231.0,103.0,LAC,108,1,2019,2018-10-19,False
11265,240.0,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
11266,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True


In [12]:
games_df.to_csv("nba_games.csv")